In [1]:
from tqdm import tqdm
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker

import numpy as np
import os, glob
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=1
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle
import argparse, subprocess
from importlib import reload
import cmasher as cmr


mode = 'nh'
iout = 1026
repo, rurmode, dp = mode2repo(mode)
snap = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snaps = uri.TimeSeries(snap)
snaps.read_iout_avail()
nout = snaps.iout_avail['iout']
gals1 = uhmi.HaloMaker.load(snap, galaxy=True, double_precision=dp)
gals2 = pklload("./database/01_nh_ghmatch.pickle")
hals = uhmi.HaloMaker.load(snap, galaxy=False, double_precision=dp)
LG = pklload(f"./database/11_LocalGroup.pickle")
for key in LG.keys():
    satids = LG[key]['sats']['id']
    ind = isin(satids, LG[key]['real']['gid'])
    omitted = satids[~ind]
    if(len(omitted)>0):
        dtype = LG[key]['real'].dtype
        arr = np.zeros(len(omitted), dtype=dtype)
        arr['gid'] = omitted
        arr['state'] = 'orphan'
        LG[key]['real'] = np.hstack((LG[key]['real'], arr))

bggs = None
for key in LG.keys():
    temp = LG[key]['BGG']
    bggs = temp if(bggs is None) else np.hstack((bggs, temp))

sats = None
for key in LG.keys():
    temp = LG[key]['sats']
    sats = temp if(sats is None) else np.hstack((sats, temp))

subs = None
for key in LG.keys():
    temp = LG[key]['subs']
    subs = temp if(subs is None) else np.hstack((subs, temp))

pairs = None
for key in LG.keys():
    temp = LG[key]['real']
    temp = temp[temp['state'] == 'pair']
    pairs = temp if(pairs is None) else np.hstack((pairs, temp))

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)


In [2]:
subids = None
for key in LG.keys():
    tmp = LG[key]['real']['hid']
    tmp = tmp[tmp>0]
    subids = tmp if(subids is None) else np.union1d(subids, tmp)
subids.shape

(2630,)

In [3]:
for subid in tqdm( subids ):
    if(os.path.exists(f"./database/submember/z017/DM_{subid:07d}.pickle")): continue
    mem = uhmi.HaloMaker.read_member_part(snap, subid, galaxy=False, target_fields=['x','y','z','id'])
    pklsave(mem.table, f"./database/submember/z017/DM_{subid:07d}.pickle")

100%|██████████| 2630/2630 [00:00<00:00, 16989.96it/s]


In [4]:
zreds = np.array([0.3, 0.5, 1, 1.5, 2, 3])
aexps = 1/(1+zreds)
iouts = np.zeros(len(zreds), dtype=np.int32)
for i in range(len(zreds)):
    arg = np.argmin(np.abs(snaps.iout_avail['aexp']-aexps[i]))
    iouts[i] = snaps.iout_avail['iout'][arg]
iouts

array([871, 717, 446, 335, 252, 158], dtype=int32)

In [5]:
for iout in iouts:
    isnap = snaps.get_snap(iout)
    dirname = f"z{round(isnap.z*100):03d}"
    if(os.path.exists(f"./database/submember/{dirname}/DM_{subids[-1]:07d}.pickle")): continue
    if(not os.path.isdir(f"./database/submember/{dirname}")):
        os.makedirs(f"./database/submember/{dirname}")
    print(f"[{iout:04d} ({dirname})]")
    isnap.get_part(pname='dm', nthread=32, target_fields=['x','y','z','id'])
    print("Get part done")
    assert (isnap.part['id']>0).all()
    argsort = np.argsort(isnap.part['id'])
    dm = isnap.part[argsort]
    print("Argsort done")

    for subid in tqdm(subids):
        if(os.path.exists(f"./database/submember/{dirname}/DM_{subid:07d}.pickle")): continue
        memid = pklload(f"./database/submember/z017/DM_{subid:07d}.pickle")['id']
        nowmem = dm[memid-1]
        pklsave(nowmem.table, f"./database/submember/{dirname}/DM_{subid:07d}.pickle")
    isnap.clear()

[Output 00871] Age (Gyr) : 10.335 / 13.741, z = 0.29925 (a = 0.7697)
[0871 (z030)]
Reading 4800 part files (33.47 GiB) in /storage6/NewHorizon/snapshots/output_00871... 


Reading parts:   0%|          | 0/4800 [00:00<?, ?it/s]

Done (6.573s).
Get part done
Argsort done


100%|██████████| 2630/2630 [00:09<00:00, 277.59it/s]


[clear] Clearing memory
	Unlink `psm_160656ad`
[Output 00717] Age (Gyr) : 8.686 / 13.741, z = 0.50068 (a = 0.6664)
[0717 (z050)]
Reading 4800 part files (39.84 GiB) in /storage6/NewHorizon/snapshots/output_00717... 


...

[clear] Clearing memory
	Unlink `psm_dda428ea`
[Output 00158] Age (Gyr) : 2.197 / 13.741, z = 3.00643 (a = 0.2496)
[0158 (z301)]
Reading 4800 part files (21.73 GiB) in /storage6/NewHorizon/snapshots/output_00158... 


Reading parts:   0%|          | 0/4800 [00:00<?, ?it/s]

Done (223.803s).
Get part done
Argsort done


100%|██████████| 2630/2630 [00:09<00:00, 277.43it/s]


[clear] Clearing memory
	Unlink `psm_e2ac1966`
